### Set up - working directory, imports...

In [ ]:
import numpy as np
import os
import torch
import matplotlib.pyplot as plt
import math
import pandas as pd

In [ ]:
os.getcwd()

In [ ]:
import os

# Specify the new directory path
project_dir = os.getcwd()+"/../"


# Change the current working directory
os.chdir(project_dir)

# Verify the change
print("Current Working Directory:", os.getcwd())


In [ ]:
# We set the randomness to 42 for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Latent dimensionality, penalty hyperparameters, and the clinical (+ histology) features to consider
L = 64
FOLDS = 10
loss_args = {'noise_factor': 0.001, 'reg_param': 0.10, 'rho': 0.001}
clinicalVars = ['MATH', 'HE_TUMOR_CELL_CONTENT_IN_TUMOR_AREA', 'PD-L1_TOTAL_IMMUNE_CELLS_PER_TUMOR_AREA',
            'CD8_POSITIVE_CELLS_TUMOR_CENTER', 'CD8_POSITIVE_CELLS_TOTAL_AREA']

COHORTS = ['Avelumab+Axitinib','Sunitinib']
BATCH_SIZES = [255, 255]

WITH_HISTOLOGY = False

if WITH_HISTOLOGY is False:
    clinicalVars = ['HE_TUMOR_CELL_CONTENT_IN_TUMOR_AREA', 'PD-L1_TOTAL_IMMUNE_CELLS_PER_TUMOR_AREA']


In [ ]:
import os

# Specify the new directory path
project_dir = os.getcwd()+"/Implementation"


# Change the current working directory
os.chdir(project_dir)

# Verify the change
print("Current Working Directory:", os.getcwd())


In [ ]:
from Logic.FoldObject import FoldObject
from Logic.Losses.LossHandler import LossHandler
from Logic.Losses.LossType import LossType
from Logic.CustomKFoldScikit import CustomKFold
from Logic.TabularDataLoader import TabularDataLoader
from Logic.Trainer import Trainer, draw_latent_space, plot_cindex, plot_coefs, plot_tsne_coefs, plot_auc, evaluate_demographic_data
from Logic.TrainingModel import TrainingModel

In [ ]:
current_directory = os.getcwd()
somepath = os.path.abspath(
    os.path.join(current_directory, '..', '..', 'P10', 'Data', 'RNA_dataset_tabular_R3.csv'))

cohort = COHORTS[1]
BATCH_SIZE = BATCH_SIZES[1]

d = TabularDataLoader(somepath, ['PFS_P', 'PFS_P_CNSR'], clinicalVars, (1/FOLDS), 0.2, BATCH_SIZE, FOLDS, cohort)
foldObject = FoldObject('PCA', FOLDS, d.allDatasets)

for fold in range(FOLDS):
    instanceModel = TrainingModel("PCA_test", d, foldObject.iterations[fold], clinicalVars,
                     None, None, None, None, BATCH_SIZE, L, False)
    

In [ ]:
# trainingmodel -> train loader -> (genetic, clinical, pfs)

ITERATION = 0

instanceModel = TrainingModel("PCA_test", d, foldObject.iterations[ITERATION], clinicalVars,
                     None, None, None, None, BATCH_SIZE, L, False)

In [ ]:
X_gen = instanceModel.train_loader[0][0].cpu()
X_cli = instanceModel.train_loader[0][1].cpu()
X_pfs = instanceModel.train_loader[0][2].cpu()

Y_gen = instanceModel.test_loader[0][0].cpu()
Y_cli = instanceModel.test_loader[0][1].cpu()
Y_pfs = instanceModel.test_loader[0][2].cpu()

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=L)

# Fit PCA to the data and transform (encode)
pca.fit(X_gen)
X_gen_encoded = pca.transform(X_gen)
Y_gen_encoded = pca.transform(Y_gen)

In [ ]:
X_gencli = np.concatenate((X_gen_encoded, X_cli), axis=1)
Y_gencli = np.concatenate((Y_gen_encoded, Y_cli), axis=1)

In [ ]:
X_pfs_transformed = np.array([(bool(event), float(time)) for event, time in X_pfs], dtype=[('event', bool), ('time', float)])
Y_pfs_transformed = np.array([(bool(event), float(time)) for event, time in Y_pfs], dtype=[('event', bool), ('time', float)])

In [ ]:
demographic_DF = pd.DataFrame()
demographic_DF['PFS_P'] = Y_pfs_transformed['time']

In [ ]:
import torch

from sklearn.cluster import KMeans
from sklearn.feature_selection import mutual_info_regression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sksurv.metrics import cumulative_dynamic_auc, as_concordance_index_ipcw_scorer
from torch.optim.lr_scheduler import StepLR

from Logic.CustomKFoldScikit import CustomKFold
from Logic.TrainingModel import TrainingModel
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
from sklearn.exceptions import FitFailedWarning
from sklearn.preprocessing import StandardScaler
import matplotlib.gridspec as gridspec

import seaborn as sns

from sklearn.manifold import TSNE
import xlsxwriter

import copy

In [ ]:

TRIES = 3
non_zero = 0
offset = 0.9

start = 0.0001
stop = 0.01
step = 0.0002

latent_cols = ["Latent " + str(x) for x in list(range(L))]
latent_cols += clinicalVars
latent_idxs = np.arange(L + len(clinicalVars))

OK = False
while not OK:
    OK = True
    estimated_alphas = np.arange(start, stop + step, step)

    # we remove warnings when coefficients in Cox PH model are 0
    warnings.simplefilter("ignore", UserWarning)
    warnings.simplefilter("ignore", FitFailedWarning)
    warnings.simplefilter("ignore", ArithmeticError)

    # we scale for better performance.
    scaler = StandardScaler()
    scaler.fit(X_gencli)
    scaled_latent_space_train = scaler.transform(X_gencli)
    scaled_latent_space_test = scaler.transform(Y_gencli)

    # We perform grid search to find the best alpha to test with
    cv = CustomKFold(n_splits=7, shuffle=True, random_state=40)
    gcv = GridSearchCV(
        as_concordance_index_ipcw_scorer(
            CoxnetSurvivalAnalysis(l1_ratio=0.5, fit_baseline_model=True, max_iter=80000, normalize=False)),
        param_grid={"estimator__alphas": [[v] for v in estimated_alphas]},
        cv=cv,
        error_score=0,
        n_jobs=5,
    ).fit(scaled_latent_space_train, X_pfs_transformed)

    cv_results = pd.DataFrame(gcv.cv_results_)

    alphas = cv_results.param_estimator__alphas.map(lambda x: x[0])
    mean = cv_results.mean_test_score
    std = cv_results.std_test_score

    best_model = gcv.best_estimator_.estimator
    best_coefs = pd.DataFrame(best_model.coef_, index=latent_cols, columns=["coefficient"])
    best_alpha = gcv.best_params_["estimator__alphas"][0]

In [ ]:
plot_cindex(alphas, mean, std, best_alpha, "pca_c-index")

non_zero = np.sum(best_coefs.iloc[:, 0] != 0)
print(f"Number of non-zero coefficients: {non_zero}")

survival_functions_tmp = best_model.predict_survival_function(scaled_latent_space_test, best_alpha)

# if the coefs are all zero OR if for some reason the survival functions couldn't be estimated, try again
if non_zero == 0 or np.isnan(survival_functions_tmp[0].y).any():
    OK = False
    TRIES -= 1
    start *= offset
    step *= offset
    stop *= offset
    print("All coefficients are 0 or survival functions are undefined... Tries left: " + str(
        TRIES) + " with start: " + str(start))
    if TRIES == 0:
        print("FAIL!")

non_zero_coefs = best_coefs.query("coefficient != 0")
coef_order = non_zero_coefs.abs().sort_values("coefficient").index

# we plot a figure showing how much weight each coef has
plot_coefs(non_zero_coefs, coef_order, "relevant_features")

latent_data = zip(latent_cols, latent_idxs)
idxs_interest = []
cols_interest = list(coef_order)

for col, idx in latent_data:
    if col in list(coef_order):
        idxs_interest += [idx]

data_points = X_gencli[:, idxs_interest]

# if we have more than 2 coefs, we perform  tsne. However this isn't uesful in our work.
if non_zero >= 2:
    plot_tsne_coefs(data_points, cols_interest, "tsne")

# We keep the best 5 coefficients for the correlation plots (check method for more info)
latent_data = zip(latent_cols, latent_idxs)
best_coefs = coef_order[-5:]
best_indices = [idx for col, idx in latent_data if col in best_coefs]

print("Best index is", best_indices)
data_points_best_coef = np.array(X_gencli)[:, best_indices]


# Predict using the best model and the test latent space
cph_risk_scores = best_model.predict(scaled_latent_space_test, alpha = best_alpha)

times = Y_pfs_transformed['time']

va_times = np.arange(min(times), max(times), 0.5)
cph_auc, _ = cumulative_dynamic_auc(X_pfs_transformed, Y_pfs_transformed, cph_risk_scores, va_times)

# we plot the Area under ROC
meanRes = plot_auc(va_times, cph_auc, "ROC")

# Using survival functions, obtain median OR mean and assign it to each patient.
survival_functions = best_model.predict_survival_function(scaled_latent_space_test, best_alpha)
predicted_times = []


# we can either use mean or median to predict PFS with.
mode = "Mean"
print("Using prediction mode: ", mode)

if mode == "Mean":
    for g in range(len(survival_functions)):
        mean_value = np.trapz(survival_functions[g].y, survival_functions[g].x) # area under survival function
        predicted_times += [mean_value]
elif mode == "Median":
    for g in range(len(survival_functions)):
        median_value = np.interp(0.5, survival_functions[g].y[::-1], survival_functions[g].x[::-1])
        predicted_times += [median_value]

demographic_DF['predicted_PFS'] = predicted_times

# we get the overall plot that shows how the model performed with the predictions
mseError = evaluate_demographic_data("PCA_test", survival_functions, demographic_DF)
# we obtain the percentage of overestimation in our PFS predictions
percentageOverEstimation = (demographic_DF['predicted_PFS'] > demographic_DF['PFS_P']).mean() * 100